Objective: Minimize risk while meeting goals

Goals / Constraints:

    -Minimum return at least 7.5%
    
    -At least 50% of investment A rating
    
    -At least 40% of investment immediately liquid
    
    -max in savings and cd = $30,000

In [1]:
from pulp import *
import pandas as pd
df = pd.read_excel(r"C:\Users\Andrew\Desktop\Fin_optimization.xlsx")
df

,Designation,Potential Investment,Expected Return,Rating,Risk,Liquidity
0,x1,Savings Account,0.040,A,0,Immediate
1,x2,Certificate of Deposit,0.052,A,0,5-year
2,x3,Atlantic Lighting,0.071,B+,25,Immediate
3,x4,Arkansas REIT,0.100,B+,30,Immediate
4,x5,Bedrock Insurance Annuity,0.082,A,20,1-year
5,x6,Nocal Mining Bond,0.065,B+,15,1-year
6,x7,Minocomp Systems,0.200,A,65,Immediate
7,x8,Antony Hotels,0.125,C,40,Immediate


In [2]:
#1's and 0's
df['Liquidity'] = (df['Liquidity']=='Immediate')
df['Liquidity'] = df['Liquidity'].astype(int)

#1's and 0's
df['Rating'] = (df['Rating']=='A')
df['Rating']= df['Rating'].astype(int)

#1's and 0's
savecd = [1,1,0,0,0,0,0,0]
df['Saving&CD'] = savecd

#1's and 0's
amt_invested = [1]*8
df['Amt_Invested'] = amt_invested
df

,Designation,Potential Investment,Expected Return,Rating,Risk,Liquidity,Saving&CD,Amt_Invested
0,x1,Savings Account,0.040,1,0,1,1,1
1,x2,Certificate of Deposit,0.052,1,0,0,1,1
2,x3,Atlantic Lighting,0.071,0,25,1,0,1
3,x4,Arkansas REIT,0.100,0,30,1,0,1
4,x5,Bedrock Insurance Annuity,0.082,1,20,0,0,1
5,x6,Nocal Mining Bond,0.065,0,15,0,0,1
6,x7,Minocomp Systems,0.200,1,65,1,0,1
7,x8,Antony Hotels,0.125,0,40,1,0,1


In [3]:
prob = LpProblem("Portfolio_Opt",LpMinimize)

In [4]:
prob

Portfolio_Opt:
MINIMIZE
None
VARIABLES

In [5]:
# Create a list of the inventment items
inv_items = list(df['Potential Investment'])

# Create a dictinary of risks for all inv items
risks = dict(zip(inv_items,df['Risk']))

# Create a dictionary of returns for all inv items
returns = dict(zip(inv_items,df['Expected Return']))

#Create dictionary for ratings of inv items
ratings = dict(zip(inv_items,df['Rating']))

# Create a dictionary for liquidity for all inv items
liquidity = dict(zip(inv_items,df['Liquidity']))

#Create a dictionary for savecd for inve items
savecd = dict(zip(inv_items,df['Saving&CD']))

#Create a dictionary for amt as being all 1's
amt = dict(zip(inv_items,df['Amt_Invested']))
risks

{'Savings Account': 0,
 'Certificate of Deposit': 0,
 'Atlantic Lighting': 25,
 'Arkansas REIT': 30,
 'Bedrock Insurance Annuity': 20,
 'Nocal Mining Bond': 15,
 'Minocomp Systems': 65,
 'Antony Hotels': 40}

In [6]:
inv_vars = LpVariable.dicts("Potential Investment",inv_items,lowBound=0,cat='Continuous')
inv_vars

{'Savings Account': Potential_Investment_Savings_Account,
 'Certificate of Deposit': Potential_Investment_Certificate_of_Deposit,
 'Atlantic Lighting': Potential_Investment_Atlantic_Lighting,
 'Arkansas REIT': Potential_Investment_Arkansas_REIT,
 'Bedrock Insurance Annuity': Potential_Investment_Bedrock_Insurance_Annuity,
 'Nocal Mining Bond': Potential_Investment_Nocal_Mining_Bond,
 'Minocomp Systems': Potential_Investment_Minocomp_Systems,
 'Antony Hotels': Potential_Investment_Antony_Hotels}

In [7]:
prob += lpSum([risks[i]*inv_vars[i] for i in inv_items])
prob

Portfolio_Opt:
MINIMIZE
40*Potential_Investment_Antony_Hotels + 30*Potential_Investment_Arkansas_REIT + 25*Potential_Investment_Atlantic_Lighting + 20*Potential_Investment_Bedrock_Insurance_Annuity + 65*Potential_Investment_Minocomp_Systems + 15*Potential_Investment_Nocal_Mining_Bond + 0
VARIABLES
Potential_Investment_Antony_Hotels Continuous
Potential_Investment_Arkansas_REIT Continuous
Potential_Investment_Atlantic_Lighting Continuous
Potential_Investment_Bedrock_Insurance_Annuity Continuous
Potential_Investment_Minocomp_Systems Continuous
Potential_Investment_Nocal_Mining_Bond Continuous

In [8]:
# amt
prob += lpSum([amt[f] * inv_vars[f] for f in inv_items]) == 100000, "Investments"
prob += lpSum([returns[f] * inv_vars[f] for f in inv_items]) >= 7500, "Returns"
prob += lpSum([ratings[f] * inv_vars[f] for f in inv_items]) >= 50000, "Ratings"
prob += lpSum([liquidity[f] * inv_vars[f] for f in inv_items]) >= 40000, "Liquidity"
prob += lpSum([savecd[f] * inv_vars[f] for f in inv_items]) <= 30000, "Save and CD"

In [9]:
prob

Portfolio_Opt:
MINIMIZE
40*Potential_Investment_Antony_Hotels + 30*Potential_Investment_Arkansas_REIT + 25*Potential_Investment_Atlantic_Lighting + 20*Potential_Investment_Bedrock_Insurance_Annuity + 65*Potential_Investment_Minocomp_Systems + 15*Potential_Investment_Nocal_Mining_Bond + 0
SUBJECT TO
Investments: Potential_Investment_Antony_Hotels
 + Potential_Investment_Arkansas_REIT + Potential_Investment_Atlantic_Lighting
 + Potential_Investment_Bedrock_Insurance_Annuity
 + Potential_Investment_Certificate_of_Deposit
 + Potential_Investment_Minocomp_Systems
 + Potential_Investment_Nocal_Mining_Bond
 + Potential_Investment_Savings_Account = 100000

Returns: 0.125 Potential_Investment_Antony_Hotels
 + 0.1 Potential_Investment_Arkansas_REIT
 + 0.071 Potential_Investment_Atlantic_Lighting
 + 0.082 Potential_Investment_Bedrock_Insurance_Annuity
 + 0.052 Potential_Investment_Certificate_of_Deposit
 + 0.2 Potential_Investment_Minocomp_Systems
 + 0.065 Potential_Investment_Nocal_Mining_Bond
 

In [11]:
prob.solve()

1

In [12]:
prob.writeLP("Portfolio_Opt.lp")
print("The optimal portfolio consists of\n"+"-"*110)
for v in prob.variables():
    if v.varValue>0:
        print(v.name, "=", v.varValue)

The optimal portfolio consists of
--------------------------------------------------------------------------------------------------------------
Potential_Investment_Arkansas_REIT = 22666.667
Potential_Investment_Bedrock_Insurance_Annuity = 47333.333
Potential_Investment_Certificate_of_Deposit = 12666.667
Potential_Investment_Savings_Account = 17333.333
